<a href="https://colab.research.google.com/github/cawllol/Deep-learning/blob/master/TensorFlow_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
#as_supervised is loading data in a 2 tuple structure[input,target]
#with_info provides a tuple containing info about version,features
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
#train,test,validation
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

#scalling
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

#shuffelling
BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
#for making same shape & object property same as training data
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.


In [0]:
#outline model
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')   
                            ])

In [0]:
#optimizer & loss function
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
#training
NUM_EPOCHS = 5
model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 9s - loss: 0.4159 - accuracy: 0.8813 - val_loss: 0.2094 - val_accuracy: 0.9397
Epoch 2/5
540/540 - 9s - loss: 0.1792 - accuracy: 0.9484 - val_loss: 0.1584 - val_accuracy: 0.9562
Epoch 3/5
540/540 - 9s - loss: 0.1376 - accuracy: 0.9599 - val_loss: 0.1325 - val_accuracy: 0.9605
Epoch 4/5
540/540 - 9s - loss: 0.1125 - accuracy: 0.9669 - val_loss: 0.1064 - val_accuracy: 0.9708
Epoch 5/5
540/540 - 8s - loss: 0.0964 - accuracy: 0.9708 - val_loss: 0.0953 - val_accuracy: 0.9703


In [6]:
#test the model
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 2ms/step - loss: 0.1065 - accuracy: 0.9681


In [7]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.11. Test accuracy: 96.81%
